## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-04-14-54-54 +0000,nyt,"Yasser Abu Shabab, Militant Leader Backed by I...",https://www.nytimes.com/2025/12/04/world/middl...
1,2025-12-04-14-54-12 +0000,nypost,Marine killed during training at Camp Pendleto...,https://nypost.com/2025/12/04/us-news/marine-k...
2,2025-12-04-14-53-21 +0000,nypost,Chrysler recalls 72K Ram pickup trucks over fa...,https://nypost.com/2025/12/04/business/chrysle...
3,2025-12-04-14-49-57 +0000,wapo,England warns people to stop visiting ER for i...,https://www.washingtonpost.com/world/2025/12/0...
4,2025-12-04-14-47-50 +0000,bbc,Crypto investor donates record £9m to Reform UK,https://www.bbc.com/news/articles/c30j8r034y8o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
216,trump,58
88,new,19
56,he,14
148,putin,14
224,ukraine,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
211,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...,107
128,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...,104
31,2025-12-04-13-29-44 +0000,wapo,Putin maintains hard line in talks as Trump sa...,https://www.washingtonpost.com/world/2025/12/0...,101
30,2025-12-04-13-34-02 +0000,cbc,"Trump could leave trade deal next year, top tr...",https://www.cbc.ca/news/world/livestory/canada...,100
256,2025-12-03-22-33-23 +0000,bbc,Trump administration to roll back fuel economy...,https://www.bbc.com/news/articles/cly13we8zxeo...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
211,107,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...
215,86,2025-12-04-00-00-00 +0000,wsj,Defense Secretary Pete Hegseth shocked officia...,https://www.wsj.com/politics/national-security...
103,64,2025-12-04-10-39-00 +0000,wsj,"New York Times Sues Hegseth, Defense Departmen...",https://www.wsj.com/business/media/new-york-ti...
72,54,2025-12-04-11-53-24 +0000,nypost,Admiral who ordered September boat strike will...,https://nypost.com/2025/12/04/us-news/admiral-...
193,48,2025-12-04-01-55-16 +0000,nypost,Halle Berry bashes Gov. Gavin Newsom at NYT su...,https://nypost.com/2025/12/03/us-news/halle-be...
128,46,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...
328,45,2025-12-03-19-37-05 +0000,latimes,Former sheriff's deputy killed after police ch...,https://www.latimes.com/california/story/2025-...
187,43,2025-12-04-02-31-07 +0000,nyt,What to Know About Putin’s Visit to India as O...,https://www.nytimes.com/2025/12/03/world/asia/...
86,42,2025-12-04-11-02-24 +0000,bbc,Israel says Palestinian groups returned body o...,https://www.bbc.com/news/articles/cd6x2j4g3z7o...
132,42,2025-12-04-09-53-02 +0000,nypost,Missing high school coach Travis Turner left h...,https://nypost.com/2025/12/04/sports/missing-h...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
